# CS572 Project
* We selected the paper: {}
* Based on the data and model of given paper, we are planning to make an extension of LSTM that predicts {}.

## Preparing

In [4]:
import torch

Check if your computer is prepared to run pytorch model with CUDA.

In [12]:
if torch.rand(1, 1) and torch.cuda.is_available():
    print("OK")

OK
